This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4550 N 27TH ST, Milwaukee, WI - [43.10012582653357, -87.94667244460172]
Parsed: 5357 N 38TH ST, Milwaukee, WI - [43.11501286372555, -87.96002113200394]
Parsed: 3500 W FLORIST AV, Milwaukee, WI - [43.12658602141596, -87.95617649105935]
Parsed: 2510 W CAPITOL DR, Milwaukee, WI - [43.08985048600596, -87.94515116472387]
Parsed: 4026 N 25TH ST, Milwaukee, WI - [43.090469245628725, -87.94462537910455]
Parsed: 2300 W CAPITOL DR, Milwaukee, WI - [43.08982951960866, -87.94096693360511]
Parsed: 4911 N 50TH ST, Milwaukee, WI - [43.106715754371294, -87.9751781281088]
Parsed: 4758 N 31ST ST, Milwaukee, WI - [43.10378966472388, -87.9516428280341]
Parsed: 1634 W CAPITOL DR, Milwaukee, WI - [43.08957951154119, -87.93186322009349]
Parsed: 4339 N 27TH ST, Milwaukee, WI - [43.0960950313636, -87.94685861036386]
Parsed: 1801 W OLIVE ST, Milwaukee, WI - [43.09399677367502, -87.93350891822978]
Parsed: 4628 N 29TH ST, Milwaukee, WI - [43.10152132944776, -87.94907235356932]
Parsed: 6044 N 36TH ST, Milw

Parsed: 330 N GLENVIEW AV, Milwaukee, WI - [43.034417046627055, -88.01649860768082]
Parsed: 6433 W LOCUST ST, Milwaukee, WI - [43.071647481245456, -87.99304049999999]
Parsed: 4831 W STATE ST, Milwaukee, WI - [43.04362145043011, -87.97366044950648]
Parsed: 100 N MITCHELL BL, Milwaukee, WI - [43.028781976348085, -87.97877566764105]
Parsed: 2760 N 55TH ST, Milwaukee, WI - [43.06935613627445, -87.98206291185313]
Parsed: 100 N MITCHELL BL, Milwaukee, WI - [43.028781976348085, -87.97877566764105]
Parsed: 6331 W KEEFE AVENUE PK, Milwaukee, WI - [43.0824357627227, -87.99128486509498]
Parsed: 5927 W APPLETON AV, Milwaukee, WI - [43.070109204097314, -87.98678824238247]
Parsed: 2674 N 56TH ST, Milwaukee, WI - [43.06751678130756, -87.9832760589463]
Parsed: 2751 N 57TH ST, Milwaukee, WI - [43.06910483819033, -87.98428762089544]
Parsed: 3050 N 54TH ST, Milwaukee, WI - [43.074602413266774, -87.98081288631792]
Parsed: 2367 N 52ND ST, Milwaukee, WI - [43.06184233680771, -87.97877288826592]
Parsed: 2033

Parsed: 935 S 5TH ST, Milwaukee, WI - [43.02170798889149, -87.91691811815414]
Parsed: 2679 S 15TH ST, Milwaukee, WI - [42.99595250291421, -87.93136857372015]
Parsed: 1308 W WALKER ST, Milwaukee, WI - [43.02221052986305, -87.92836191618096]
Parsed: 800 S 15TH ST, Milwaukee, WI - [43.02307576019321, -87.93049055471057]
Parsed: 222 S 3RD ST, Milwaukee, WI - [43.02870199417163, -87.91393540074463]
Parsed: 1211 W ROGERS ST, Milwaukee, WI - [43.008239474032095, -87.92743066763806]
Parsed: 222 S 3RD ST, Milwaukee, WI - [43.02870199417163, -87.91393540074463]
Parsed: 1669 S 6TH ST, Milwaukee, WI - [43.012711760199664, -87.91856051543633]
Parsed: 900 W WALKER ST, Milwaukee, WI - [43.022229525967916, -87.92269161226845]
Parsed: 1550 W CLEVELAND AV, Milwaukee, WI - [42.995627453257384, -87.93274352262104]
Parsed: 1543 S MUSKEGO AV, Milwaukee, WI - [43.015071445755574, -87.935133728028]
Parsed: 1800 W GREENFIELD AV, Milwaukee, WI - [43.017039511541185, -87.93575150291419]
Parsed: 560 W BECHER ST, 

Parsed: 5865 S 13TH ST #4, Milwaukee, WI - [42.937875167638055, -87.92956805539828]
Parsed: 4747 S HOWELL AV, Milwaukee, WI - [42.957313805441736, -87.90981854053793]
Parsed: 4747 S HOWELL AV, Milwaukee, WI - [42.957313805441736, -87.90981854053793]
Parsed: 5311 S HOWELL AV, Milwaukee, WI - [42.94836660115995, -87.90988156217801]
Parsed: 5800 S HONEY CREEK DR, Milwaukee, WI - [42.93960053733623, -87.95922858211692]
Parsed: 3815 S 5TH PL, Milwaukee, WI - [42.97514450291419, -87.91793157372014]
Parsed: 4108 S 5TH PL, Milwaukee, WI - [42.96994272300768, -87.91807291185313]
Parsed: 4569 S 20TH ST, Milwaukee, WI - [42.96144638773154, -87.93893951442578]
Parsed: 1400 W ZELLMAN CT, Milwaukee, WI - [42.92520249379625, -87.93143163918865]
Parsed: 2229 S 17TH ST, Milwaukee, WI - [43.00398125728549, -87.93445851875455]
Parsed: 1539 W IONA TR, Milwaukee, WI - [42.93846653952925, -87.9330915]
Parsed: 6160 S 6TH ST #W-27, Milwaukee, WI - [42.93318845517774, -87.91968103508366]
Parsed: 4747 S HOWELL 

Parsed: 3527 W NORTH AV, Milwaukee, WI - [43.060547790558545, -87.95787811871742]
Parsed: 2328 N 39TH ST, Milwaukee, WI - [43.06109769515457, -87.96222565213085]
Parsed: 2568 N 35TH ST, Milwaukee, WI - [43.06577291035259, -87.95741589353128]
Parsed: 1645 N 36TH ST, Milwaukee, WI - [43.05240883236195, -87.95883908425172]
Parsed: 2343 N 50TH ST #10, Milwaukee, WI - [43.061594374689825, -87.97623056650677]
Parsed: 954 N 37TH ST #311, Milwaukee, WI - [43.04280554954124, -87.9595859112762]
Parsed: 2538 N 16TH ST, Milwaukee, WI - [43.06496341326678, -87.93252487910455]
Parsed: 1909 N 32ND ST, Milwaukee, WI - [43.055397251457094, -87.95402810646873]
Parsed: 2141 N 16TH ST, Milwaukee, WI - [43.05889761226845, -87.93268159925536]
Parsed: 2141 N 16TH ST, Milwaukee, WI - [43.05889761226845, -87.93268159925536]
Parsed: 2548 N 20TH ST, Milwaukee, WI - [43.06518, -87.93750640074464]
Parsed: 2101 N 38TH ST, Milwaukee, WI - [43.057602586733225, -87.9611705809335]
Parsed: 2450 N 49TH ST, Milwaukee, WI 

...

Parsed: 5711 N 32ND ST, Milwaukee, WI - [43.12127775437128, -87.95260063200395]
Parsed: 5150 N 32ND ST, Milwaukee, WI - [43.111298535662776, -87.95270767164877]
Parsed: 4900 N HOPKINS ST, Milwaukee, WI - [43.106670659732224, -87.96088000802142]
Parsed: 1923 W FIEBRANTZ AV, Milwaukee, WI - [43.09140248124546, -87.93513394463038]
Parsed: 4162 N 17TH ST, Milwaukee, WI - [43.092917329447744, -87.93189436078269]
Parsed: 5150 N 32ND ST, Milwaukee, WI - [43.111298535662776, -87.95270767164877]
Parsed: 2819 W CARMEN AV, Milwaukee, WI - [43.12271252842076, -87.94799133527613]
Parsed: 5026 W FAIRMOUNT AV, Milwaukee, WI - [43.10837992119433, -87.97547698434117]
Parsed: 2727 W ROOSEVELT DR, Milwaukee, WI - [43.09509849567218, -87.94779283236194]
Parsed: 3610 W GREEN TREE RD, Milwaukee, WI - [43.141290312080564, -87.95650422653468]
Parsed: 5710 N 34TH ST, Milwaukee, WI - [43.12170963918865, -87.9549053280341]
Parsed: 4808 N HOPKINS ST, Milwaukee, WI - [43.105152970858114, -87.96015143903314]
Parsed

Parsed: 1015 E OTJEN ST, Milwaukee, WI - [42.99843839993658, -87.89618468497898]
Parsed: 2416 S KINNICKINNIC AV, Milwaukee, WI - [43.00073082044538, -87.90236891822978]
Parsed: 3063 S 13TH ST, Milwaukee, WI - [42.988881542001515, -87.92885188704764]
Parsed: 3232 S LOGAN AV, Milwaukee, WI - [42.98567371717931, -87.89682295902844]
Parsed: 433 E NORWICH ST, Milwaukee, WI - [42.97184753952927, -87.90375111226845]
Parsed: 245 W LINCOLN AV, Milwaukee, WI - [43.002832487881896, -87.913584]
Parsed: 3448 S TAYLOR AV, Milwaukee, WI - [42.981731999999994, -87.90050345960536]
Parsed: 570 W HOLT AV, Milwaukee, WI - [42.98178630427701, -87.91860233070935]
Parsed: 3457 S 9TH ST, Milwaukee, WI - [42.98160586372555, -87.92284901543633]
Parsed: 3423 S CLEMENT AV, Milwaukee, WI - [42.98174952844943, -87.89444008093349]
Parsed: 2855 S LENOX ST, Milwaukee, WI - [42.99255950291419, -87.89809107372014]
Parsed: 921 E OHIO AV, Milwaukee, WI - [42.98462648124546, -87.89744291909516]
Parsed: 1727 N 12TH ST, Milw

Parsed: 5800 N 80TH ST, Milwaukee, WI - [43.123241487931075, -88.01083380031343]
Parsed: 5936 N 84TH ST, Milwaukee, WI - [43.125857717179315, -88.01572540074464]
Parsed: 8832 W SILVER SPRING DR, Milwaukee, WI - [43.11963246047075, -88.0230430582838]
Parsed: 8123 W VILLARD AV, Milwaukee, WI - [43.112204488458815, -88.01254349999999]
Parsed: 4526 N 76TH ST, Milwaukee, WI - [43.0996322200935, -88.00691637131426]
Parsed: 2857 N FRATNEY ST, Milwaukee, WI - [43.07055459256159, -87.90156851543632]
Parsed: 2818 N WEIL ST, Milwaukee, WI - [43.06971672300767, -87.89911240074464]
Parsed: 1300 E BRADY ST, Milwaukee, WI - [43.05298146768412, -87.89454352553523]
Parsed: 2507 N OAKLAND AV, Milwaukee, WI - [43.06386658673321, -87.88808162421367]
Parsed: 2507 N OAKLAND AV, Milwaukee, WI - [43.06386658673321, -87.88808162421367]
Parsed: 1200 N CASS ST, Milwaukee, WI - [43.045778488693, -87.90219479952843]
Parsed: 1303 N CASS ST, Milwaukee, WI - [43.04710867055226, -87.90226962479058]
Parsed: 2861 N FARW

Parsed: 3842 N 24TH PL, Milwaukee, WI - [43.08699613627445, -87.94351489684949]
Parsed: 3621 N 13TH ST, Milwaukee, WI - [43.08392758673324, -87.92723015032581]
Parsed: 1715 N 1ST ST, Milwaukee, WI - [43.05276318026244, -87.91133420861128]
Parsed: 3106 N 7TH ST, Milwaukee, WI - [43.075065782676475, -87.91999441185312]
Parsed: 3901 N 3RD ST, Milwaukee, WI - [43.08833795337296, -87.91359154097155]
Parsed: 2921 N 16TH ST, Milwaukee, WI - [43.07187536081136, -87.93322910646873]
Parsed: 3652 N 13TH ST, Milwaukee, WI - [43.08454741326679, -87.92714491185313]
Parsed: 3707 N RICHARDS ST, Milwaukee, WI - [43.08423010796928, -87.90741963315779]
Parsed: 3707 N RICHARDS ST, Milwaukee, WI - [43.08423010796928, -87.90741963315779]
Parsed: 3707 N RICHARDS ST, Milwaukee, WI - [43.08423010796928, -87.90741963315779]
Parsed: 3335 N 20TH ST, Milwaukee, WI - [43.079552947544585, -87.93730963200395]
Parsed: 3413 N MLK DR, Milwaukee, WI - [43.08076938383641, -87.91757490593784]
Parsed: 1939 N MARTIN L KING J

Parsed: 5701 W GOOD HOPE RD, Milwaukee, WI - [43.148424537798505, -87.98356138343239]
Parsed: 7869 N 60TH ST, Milwaukee, WI - [43.160519234491545, -87.98498067210919]
Parsed: 7100 N 60TH ST, Milwaukee, WI - [43.147205245628726, -87.98546387795072]
Parsed: 5800 N 64TH ST, Milwaukee, WI - [43.123051208283925, -87.99097405578637]
Parsed: 6665 N 60TH ST, Milwaukee, WI - [43.13938363780368, -87.98571762450027]
Parsed: 6942 N RAINTREE DR, Milwaukee, WI - [43.14385713209085, -87.98313696309647]
Parsed: 8975 W BROWN DEER RD, Milwaukee, WI - [43.177607531738985, -88.0230102485429]
Parsed: 8500 W BROWN DEER RD, Milwaukee, WI - [43.17797392857547, -88.01615478493345]
Parsed: 7620 W DONNA CT #2, Milwaukee, WI - [43.17175751543633, -88.00605103664373]
Parsed: 6939 N 78TH CT, Milwaukee, WI - [43.14443399999999, -88.00822169028775]
Parsed: 8115 N 107TH ST, Milwaukee, WI - [43.1651200025186, -88.04456618922391]
Parsed: 6951 N 60TH ST, Milwaukee, WI - [43.144624005828376, -87.98559011036386]
Parsed: 65